In [200]:
import os
import numpy as np
import random
import copy
from collections import defaultdict, namedtuple
os.chdir("/Users/davidamat/Documents/projects/2048/")

In [452]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tensorboardX import SummaryWriter

In [7]:
import matplotlib.pyplot as plt

In [453]:
import sys

sys.path

['/Users/davidamat/Documents/projects/2048/src',
 '/Users/davidamat/Documents/projects/2048/src',
 '/Users/davidamat/Documents/Glovo/missing-items-pipeline',
 '/Users/davidamat/opt/anaconda3/lib/python38.zip',
 '/Users/davidamat/opt/anaconda3/lib/python3.8',
 '/Users/davidamat/opt/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/Users/davidamat/.local/share/virtualenvs/2048-TNu5m-4D/lib/python3.8/site-packages',
 '/Users/davidamat/.local/share/virtualenvs/2048-TNu5m-4D/lib/python3.8/site-packages/IPython/extensions',
 '/Users/davidamat/.ipython']

In [9]:
from common import constants as c

In [169]:
class Movements():
    
    @staticmethod
    def displacement_numbers(mat):
        """
        Desplaza todos los numeros a la izquierda de la matriz
        """
        
        new = np.zeros(mat.shape) # crea una matrix de 0 de AxA (donde A es la c.GRID_LEN)
        done = False
        for i in range(c.GRID_LEN):
            count = 0
            for j in range(c.GRID_LEN):
                # Si en esa fila hay un elemento no nulo
                if mat[i][j] != 0:
                    # Pone ese elemento en la posicion count (columna), donde esta vendra determinada por si
                    # en esa fila, previamente, hemos encotrado algun otro elemento NO nulo
                    new[i][count] = mat[i][j]

                    # Solo con que modifiquemos la posicion de una de los numeros, ya contara como un movimiento: done = True
                    if j != count:
                        done = True

                    # Suma al count una posicion ya que ya se ha movido a la izquerda de todo (columna 0) ese elemento
                    count += 1
        return (new, done)
    
    @staticmethod
    def merge_numbers(mat, game_score = 0):
        """
        Suma los números de cada fila que sean iguales 
        y consecutivos, dejando el segundo sumando a 0
        """
        done = False
        for i in range(c.GRID_LEN):
            for j in range(c.GRID_LEN-1): # la ultima columna j, no tiene una columna a su derecha (j+1)
                celda = mat[i][j]
                celda_derecha = mat[i][j+1]
                if celda == celda_derecha and celda != 0: # si son iguales, y esta igualdad son numeros > 0, se suman
                    merge_val = celda + celda_derecha
                    mat[i][j] = merge_val
                    game_score += merge_val #suma los puntos
                    mat[i][j+1] = 0 # se deja la de la derecha vacía, esto hace que se necesite hacer otro displacement para llenar ese hueco
                    # esto tambien hace que al leer la siguiente columna, se lea un 0, y no el numero que estaba
                    done = True
        return (mat, done, game_score)
    
    @staticmethod
    def perform_movement(game, game_score):
        """
        1 - mover al maximo a la izquierda todos los numeros
        2 - Sumamos los iguales dejando 0 en el segundo sumando
        3 - Por seguridad, por si hemos dejado alguno sin desplazar a la izquierda (huecos generados por el merge),
            se vuelve a aplicar sin importar el done o no (no importa si mueve a alguien o no ahora)
        """
        game_disp, done_disp = Movements.displacement_numbers(game)
        game_merged, done_merge, game_score = Movements.merge_numbers(game_disp, game_score)
        game_final = Movements.displacement_numbers(game_merged)[0]
        return (game_final, done_disp or done_merge, game_score)
    
    @staticmethod
    def ro(mat, cw = True, num = 1): #cw: clockwise: True or False, #num: number of rotations
        """
        Rota 90º las matrices para que las operaciones UP, DOWN y RIGHT se puedan hacer con la de LEFT
        """
        param_clockise = (1,0) if cw else (0,1) #clockwise or counter-clockwise (see help(np.rot90))

        # Cuantas rotaciones hacemos
        rot_mat = mat
        for _ in range(num):
            rot_mat = np.rot90(np.array(rot_mat), axes = param_clockise)

        return rot_mat

    @staticmethod
    def left(game, game_score = 0):
        return Movements.perform_movement(game, game_score)
    
    @staticmethod
    def down(game, game_score = 0):
        """
        C - L - UC
        """
        rotate_game = Movements.ro(game) #rotate clockwise
        left_game, done, game_score = Movements.left(rotate_game, game_score) #apply left
        game_final = Movements.ro(left_game, cw = False) #undo the rotation
        return game_final, done, game_score
    
    @staticmethod
    def up(game, game_score = 0):
        """
        UC - L - C
        """
        rotate_game = Movements.ro(game, cw = False) #rotate anti-clockwise
        left_game, done, game_score = Movements.left(rotate_game, game_score) #apply left
        game_final = Movements.ro(left_game) #undo the rotation
        return game_final, done, game_score

    @staticmethod
    def right(game, game_score = 0):
        """
        C - C - L - UC - UC
        """
        rotate_game = Movements.ro(game,cw=True, num =2) #double rotation
        left_game, done, game_score = Movements.left(rotate_game, game_score) #apply left
        game_final = Movements.ro(left_game, cw = False, num = 2) #undo the rotation
        return game_final, done, game_score

    

In [170]:
class Env():
    
    def __init__(self, grid_size):
        """
        grid_size: size of the matrix
        """
        # Status
        self.game_stat = -2 # 0: playing, 1: win, -1: lost, -2: ready to start
        
        # Actions
        self.actions = {0: Movements.up, 
                        1: Movements.down, 
                        2: Movements.left, 
                        3: Movements.right}
        
        # Log in a list all the matrices in each step
        self.log = defaultdict(list)
        
        # Initialize matrix
        self._init_matrix(grid_size)
        
        # Score accumulated
        self.game_score = 0
    
        
    def _init_matrix(self, n):
        """
        Initializes the game matrix
        """
        self.matrix = np.zeros((n,n))
        self._add_two(times = 2)
        
        # Log
        self.log["mat"].append(self.matrix)
        self.log["action"].append(-1) # randomly added action
        self.log["reward"].append(0)
        
    def _add_two(self, times,  choices = c.RANDOM_NUMBER_CHOICES, probs_choices = c.PROBAB_NUMBER_CHOICES):
        """
        Add to the matrix randomly a 2 and 4
        """
        for _ in range(times):
            # choose only cells with a 0
            avail_cells = list(zip(*np.where(self.matrix==0)))
            
            # Choose the new index of the matrix
            index_sample = random.sample(avail_cells, 1)[0]
            
            # Start the game always with a 2
            if self.game_stat == -2:
                self.matrix[index_sample] = 2
                
                # Change the stat to playing
                self.game_stat = 0
            
            elif len(avail_cells):
                # Choose randomly between a 2 or a 4 and put it in the matrix
                value_sample = np.random.choice(choices, p = probs_choices)
                self.matrix[index_sample] = value_sample
            else:
                sys.exit("Finished game!")
                
    def _check_possible_action(self):
        """
         # Check if there is any possible action to take
         without modifying the env matrix
        """
        any_action_available = False
        test_matrix = copy.copy(self.matrix)
        for a_id in self.actions:
            _, action_available, _ = self.actions[a_id](test_matrix, 0)
            any_action_available |= action_available
        return any_action_available
        
        
                
    def _game_stat(self):
        """
        Status of the game:
        1: game won
        -1: game lost
        0: game in play
        """            
        if self.matrix.min() == 0:
            if self.matrix.max() >= c.OBJECTIVE:
                self.game_stat = 1
            else:
                self.game_stat = 0
        else:
            if self._check_possible_action():
                self.game_stat = 0
            else:
                self.game_stat = -1
                
        
                
    # Play step
    def step(self, action_id):
        """
        Returns:
        - start matrix
        - final matrix
        - has_moved: if the action taken has lead to a movement (True) or not (False)
        - reward
        - game stat
        """
        
        # Take the action
        start_matrix = copy.copy(self.matrix)
        self.matrix, done, reward = self.actions[action_id](self.matrix)
        self.game_score += reward
        
        # Log
        self.log["mat"].append(self.matrix)
        self.log["action"].append(action_id)
        self.log["reward"].append(reward)
        
        # If the movement could be done
        if done: 
            # Add randomly the next number in the matrix
            self._add_two(times = 1)
            
            # Log
            self.log["mat"].append(self.matrix)
            self.log["action"].append(-1) #action -1 is a randomly added number
            self.log["reward"].append(0)
            
            # Check game status if a further action is possible
            self._game_stat()
        
        # If the movement performed didn't change anything keep playing
        return start_matrix, self.matrix, done, reward, self.game_stat
    
    # Reset
    def reset(self):
        self.__init__(self.matrix.shape[0])

In [171]:
class Player():
    """
    Debugging class for a random Player without policy
    """
    
    def __init__(self, env):
        self.env = env
        self.steps = 0
        self.game_stat = 0
        self.log = defaultdict(list)
    
    def play_random(self):
        
        while self.game_stat == 0:
            self.steps += 1
            
            # action random
            action_pl = np.random.choice(list(self.env.actions.keys()))
            
            #choose a random action
            start_matrix, end_matrix, done, reward, self.game_stat = self.env.step(action_pl)
            
            # log
            self.log["mat_o"].append(start_matrix)
            self.log["action"].append(action_pl)
            self.log["reward"].append(reward)
            self.log["mat_f"].append(end_matrix)
            self.log["done"].append(done)
            self.log["game_stat"].append(self.game_stat)

In [454]:
class Model(nn.Module):
    def __init__(self, input_size, n_actions):
        super(Model, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
        )
        
        self.apply(self._init_weights)

    def forward(self, x):
        """
        Assumes x is a tensor with the matrix raveled 
        with torch.float format (see preprocess of PolicyAgent)
        """
        return self.net(x)
    
    def _init_weights(self, m):
        """
        Initializes all weights and biases to the same quantity
        to avoid initially getting stucked into a action value
        when the network is just exploring and taking the same step
        which may lead the matrix in the same corner without moving
        until another action is sampled.
        """
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            #m.bias.data.fill_(0.01)

In [654]:
class PolicyAgent():
    """
    Policy agent gets action probabilities from the model and samples actions from it
    :params num_actions: number of actions to choose from the environment
    :params model: instance of the PyTorch model (network)
    """
    # TODO: unify code with DQNAgent, as only action selector is differs.
    def __init__(self, model, num_actions, device="cpu"):
        self.model = model
        self.device = device 
        self.num_actions = num_actions
        
    def preprocess(self, state):
        return torch.tensor(state.ravel(), dtype = torch.float)
        
    @torch.no_grad()
    def get_action_probs(self, state):
        """
        Given a state matrix, get the probs of the last layer
        """
        state = self.preprocess(state).to(self.device)
        return F.softmax(self.model(state) ,dim=0).data.cpu().numpy()
        
    @torch.no_grad()
    def __call__(self, state, epsilon = 0):
        """
        Return actions from given a state
        :param state: matrix of state
        :param epsilon: epsilon value to choose from random action
        :return: action index
        """
        assert isinstance(state, np.ndarray)
        # take a random choice
        if (epsilon > 0) & (np.random.rand() < epsilon):
            
            return np.random.choice(self.num_actions) 
        
        # Forward the state to get the actions probabilities
        else:
            
            probs = self.get_action_probs(state)
            return np.random.choice(self.num_actions, p=probs)

In [655]:
class EpsilonPolicy():
    """
    Sets a decayment policy of the epsilon for the 
    agent to ignore the policy-based action
    and perform a random action instead when training
    First epochs should have higher epsilon to allow exploration
    :params eps_decay: number of epochs in which epsilon goes from eps_start to eps_decay
    """
    
    def __init__(self, eps_start, eps_decay, eps_final):
        self.eps_start = eps_start
        self.eps_final = eps_final
        self.eps_decay = eps_decay
    
    def get_epsilon(self, epoch):
        return max(self.eps_final, self.eps_start  - epoch / self.eps_decay)

In [708]:
ExperienceEpisode= namedtuple('ExperienceEpisode', ('state', 'action', 'done', 'reward',  'game_stat', 'epsilon'))

class ExperienceSource():
    """
    Helps in the REINFORCE algorithm providing
    - A continuous source of steps for 1 single episode until the buffer gets reset
    - Inputs:
        - env:
        - agent: performs actions based on a policy (REINFORCE -> on-policy)
        - epsilon_policy: instance of class EpsilonPolicy
    Returns:
        - reward: for each step 
        - state: state for each step in the episode
        - action: action decided by the agent to be taken at each step
    
    """
    def __init__(self, env, agent, epsilon_policy):
        self.env = env
        self.agent = agent
        self.epsilon_policy = epsilon_policy
        
        # Attributes
        self.history = [] # history of ExperienceSource instances for 1 entire episode
        self.steps = 0 # counter of actions performed
        self.epsilon = epsilon_policy.get_epsilon(0) #starting epsilon at epoch = 0
        self.env.reset() # Reset env

    def populate_episode(self, epoch_num):
        
        # Play until the episode finishes
        game_stat = 0
        
        while game_stat == 0:
            
            # Count steps
            self.steps += 1
                  
            # use the agent's policy to choose next action and also input the epsilon policy
            self.epsilon = self.epsilon_policy.get_epsilon(epoch_num)
            action_id = self.agent(self.env.matrix, self.epsilon)
            
            # Take the choosen action
            start_matrix, end_matrix, done, reward, game_stat = self.env.step(action_id)
                
            # fill the history of steps
            self.history.append(ExperienceEpisode(state=start_matrix, action=action_id, 
                                                  done = done, reward=reward, game_stat=game_stat, epsilon = self.epsilon))
            
    def reset(self):
        self.history = []
        self.steps = 0
        self.epsilon = self.epsilon_policy.get_epsilon(0)
        self.env.reset()

In [709]:
class QValueCalc():
    def __init__(self):
        pass
    
    def __call__(self, rewards, gamma):
        """
        Calculates the discounted total reward for every step
        rewards: list of rewards for the whole episodes
        """
        res = []
        sum_r = 0.0

        # Calculate first the reward from the end of the local reward list
        for r in reversed(rewards):

            # The more far apart we are from the last step reward, the more discounted the reward
            sum_r *= gamma

            # local reward at that timestep
            sum_r += r
            res.append(sum_r)

        # reverse again the resulting q-vals list
        return list(reversed(res))

In [728]:
if __name__== "__main__":
    
    # Constants 
    c.GRID_LEN = 4
    c.OBJECTIVE = 128
    c.GAME_WIN_RATE = 0.8
    LEARNING_RATE = 0.001
    GAMMA = 0.99
    EPOCHS = 1000000
    BATCHS = 20
    
    
    # Game initialize
    env = Env(c.GRID_LEN)
    model = Model(c.GRID_LEN**2, len(env.actions))
    eps = EpsilonPolicy(eps_start = 1, eps_decay = 100, eps_final = 0.01)
    agent = PolicyAgent(model = model, num_actions=len(env.actions))
    exp = ExperienceSource(env, agent, eps)
    qv = QValueCalc()
    
    # Training
    version = "v1"
    writer = SummaryWriter(comment=f"-2048-{version}", log_dir=f"runs/{version}")
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Log
    game_scores = [] # scores for each episode
    steps_reach = [] # steps reached for each episode
    game_wins = [] # whether 0: game lost, 1: game won
        
    # Counters
    step_idx = 0
    done_episodes = 0
    epoch_idx = 0
    
    ################
    #   Epochs
    ################
    while epoch_idx < EPOCHS:
        if ((epoch_idx % 10) == 0) & (epoch_idx > 0):
            print("Epoch: ", epoch_idx,
                  ", Game_scores_mean: " , np.mean(game_scores),
                  ", Mean wins: ", mean_wins)
        
        # For each step in the episode, keep track also of states, actions, rewards -> qvals
        batch_states, batch_actions, batch_rewards = [], [], []
        
        ###############
        #   Batchs
        ###############
        # Play several games with the same policy
        batch_episodes = 0
        
        # For each batch
        for _ in range(BATCHS):
        
            # Generate a episode
            exp.populate_episode(epoch_idx)


            # Iterate through episode
            for idx, exp_step in enumerate(exp.history):

                # Fill with experience data
                batch_states.append(exp_step.state)
                batch_actions.append(int(exp_step.action))
                batch_rewards.append(exp_step.reward)

                # convert rewards to q values according to REINFORCE
                batch_qvals = qv(batch_rewards, GAMMA)

            # Get last step number
            steps = len(exp.history)
            steps_reach.append(steps)

            # Get the final score in the episode
            game_score_final = exp.env.game_score
            game_scores.append(game_score_final)

            # Get if the game was won (1) or not (0)
            game_stat_final = 0 if exp.env.game_stat == -1 else 1
            game_wins.append(game_stat_final)       
        
        # Inform Tensorboard
        mean_rewards = float(np.mean(game_scores[-400:]))
        mean_wins = np.round(float(np.mean(game_wins[-400:])),3)
        writer.add_scalar("mean_100_scores", mean_rewards, epoch_idx)
        writer.add_scalar("game_score", game_score_final, epoch_idx)
        writer.add_scalar("steps", steps, epoch_idx)
        writer.add_scalar("mean_wins", mean_wins, epoch_idx)
        
        # When the problem is solved stop training
        if (mean_wins > c.GAME_WIN_RATE) & (epoch_idx > 20):
            break
        
        ##############################
        # Training neural network
        ##############################
        optimizer.zero_grad()
        
        # Converting to tensors the matrices of each observation in the episode
        # ----------------------------------------------------------------------
        
        # shape: [# steps, c.GRID_LEN, c.GRID_LEN]
        tensor_states = torch.FloatTensor(batch_states)
        
        # shape [# steps]
        tensor_actions = torch.LongTensor(batch_actions)
        tensor_qvals = torch.FloatTensor(batch_qvals)
        
        # Forward to the network to get logits
        # we will forwar tensor states with the following shape
        # [# steps, c.GRID_LEN * c.GRID_LEN]
        logits = model(tensor_states.view(-1, 
            torch.prod(torch.tensor(tensor_states.shape[-2:]),0).item()))
        
        # Convert logits to log_softmax
        log_softmax = F.log_softmax(logits, dim=1)
        
        # From the probabilities got, mask with the actions taken
        # log_softmax is [#steps in game, 4 (actions)] so we will
        # convert it to [# steps, 1 (action taken)]
        log_softmax_action = log_softmax.gather(1, tensor_actions.unsqueeze(1)).squeeze(1)
        
        # The loss will be the weighted sum over steps in the episode
        # of the Q values (tensor_qvals) weighting the log(policy(s,a))
        # which is the log_softmax_action
        loss = -tensor_qvals * log_softmax_action
        loss_mean = loss.mean()
        writer.add_scalar("loss", np.round(loss_mean.item(), 4), epoch_idx)
        
        # Backpropagate
        loss_mean.backward()
        optimizer.step()
        
        # Reset the experience source and add epoch counter
        exp.reset()
        epoch_idx += 1
        
    writer.close()      

Epoch:  10 , Game_scores_mean:  844.84 , Mean wins:  0.465
Epoch:  20 , Game_scores_mean:  832.91 , Mean wins:  0.4
Epoch:  30 , Game_scores_mean:  744.8933333333333 , Mean wins:  0.218
Epoch:  40 , Game_scores_mean:  762.855 , Mean wins:  0.34
Epoch:  50 , Game_scores_mean:  787.108 , Mean wins:  0.5
Epoch:  60 , Game_scores_mean:  783.0266666666666 , Mean wins:  0.41
Epoch:  70 , Game_scores_mean:  784.7114285714285 , Mean wins:  0.475
Epoch:  80 , Game_scores_mean:  792.0725 , Mean wins:  0.568
Epoch:  90 , Game_scores_mean:  780.3911111111111 , Mean wins:  0.442
Epoch:  100 , Game_scores_mean:  783.366 , Mean wins:  0.35
Epoch:  110 , Game_scores_mean:  779.9781818181818 , Mean wins:  0.358
Epoch:  120 , Game_scores_mean:  769.975 , Mean wins:  0.258
Epoch:  130 , Game_scores_mean:  767.2738461538462 , Mean wins:  0.37
Epoch:  140 , Game_scores_mean:  766.8142857142857 , Mean wins:  0.42
Epoch:  150 , Game_scores_mean:  758.1146666666667 , Mean wins:  0.3
Epoch:  160 , Game_scores_

KeyboardInterrupt: 

In [727]:
mean_wins

1.0

In [717]:
exp.reset()

In [718]:
exp.populate_episode(15100)

In [719]:
exp.history[:-10]

[ExperienceEpisode(state=array([[0., 0., 2., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 2., 0.]]), action=3, done=True, reward=0, game_stat=0, epsilon=0.01),
 ExperienceEpisode(state=array([[0., 0., 0., 2.],
        [0., 0., 0., 0.],
        [2., 0., 0., 0.],
        [0., 0., 0., 2.]]), action=1, done=True, reward=4.0, game_stat=0, epsilon=0.01),
 ExperienceEpisode(state=array([[0., 0., 0., 0.],
        [0., 0., 0., 2.],
        [0., 0., 0., 0.],
        [2., 0., 0., 4.]]), action=1, done=True, reward=0, game_stat=0, epsilon=0.01),
 ExperienceEpisode(state=array([[2., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 2.],
        [2., 0., 0., 4.]]), action=0, done=True, reward=4.0, game_stat=0, epsilon=0.01),
 ExperienceEpisode(state=array([[4., 0., 0., 2.],
        [2., 0., 0., 4.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]), action=3, done=True, reward=0, game_stat=0, epsilon=0.01),
 ExperienceEpisode(state=array([[0., 0., 4., 2.],
 